# Cluster drawing to create samples from it
Previous attemps to learn on entire drawing was unsuccessfull, so I decided to move from new approach.
I.e. i will try to create a cluster of lines from the drawing that will be a learning sample x.
Than we will calculate some distance from cluster to dimensions on the drawing to attribute each dimension with the cluster. That attribution will be our Y.

In [1]:
# Start with basic example of unsupervised clustering with unknown number of clusters
# https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-auto-examples-cluster-plot-dbscan-py

In [2]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [14]:
import pymongo
import pandas as pd
from pymongo import MongoClient

import matplotlib.pyplot as plt
%matplotlib inline

from processing import *

In [18]:
client = MongoClient('mongodb://192.168.0.104:27017')
db = client.geometry

fileids  = db.files.find({'Valid':True}).distinct('FileId')
print('files count', len(fileids))

# all the data will be too much for now, so we will take say 50 files:
fileids = fileids[:50]

query = {
    'ClassName':
    {
        '$in':[
            'AcDbLine',
            #'AcDbPolyline',
            'AcDbText',
            'AcDbRotatedDimension'
        ]
    },
    'FileId':{
        '$in':fileids
    }
}

data = pd.DataFrame(list(db.objects.find(query)))
print('data length', len(data))

files count 322
data length 164555


In [20]:
cols_to_expand = ['XLine1Point', 'XLine2Point','StartPoint','EndPoint','Position']
data = expand_columns(data, cols_to_expand)
# TODO: Explode polylines into lines

data['ClassName'] = data['ClassName'].astype('category')
data['FileId'] = data['FileId'].astype('category')

columns = [
    'ClassName', 'FileId',
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z',
                
   'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',

   'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z',
   'Position.X', 'Position.Y', 'Position.Z']
data = data[columns]

data.groupby(['FileId','ClassName']).count()

XLine1Point.X  \
FileId                               ClassName                             
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                        0.0   
                                     AcDbRotatedDimension           43.0   
...                                                                  ...   
0e5caeae-e61d-4a22-9068-a83f2d2c4242 AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
0e608710-f13d-4362-aff7-26bf466f3243 AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   

                                                           XLine1Point.Y  \
FileId                               ClassName                             
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                        0.0   
                                     AcDbRotatedDimension           43.0   
...                                                                  ...   
0e5caeae-e61d-4a22-9068-a83f2d2c4242 AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
0e608710-f13d-4362-aff7-26bf466f3243 AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   

                                                           XLine1Point.Z  \
FileId                               ClassName                             
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                        0.0   
                                     AcDbRotatedDimension           43.0   
...                                                                  ...   
0e5caeae-e61d-4a22-9068-a83f2d2c4242 AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
0e608710-f13d-4362-aff7-26bf466f3243 AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   

                                                           XLine2Point.X  \
FileId                               ClassName                             
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                        0.0   
                                     AcDbRotatedDimension           43.0   
...                                                                  ...   
0e5caeae-e61d-4a22-9068-a83f2d2c4242 AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   
0e608710-f13d-4362-aff7-26bf466f3243 AcDbLine                        0.0   
                                     AcDbRotatedDimension            NaN   
                                     AcDbText                        0.0   

                                                           XLine2Point.Y  \
FileId                     

In [ ]:
groupped = data.groupby(['FileId'])
x_columns = [
    #'ClassName', 
    'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',
    'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z']

y_columns = [
    #'ClassName', 
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z']

file_clusters = {} #dic[FileId]listOfLists
keys = list(groupped.groups.keys()
for key in keys:
    x = groupped.get_group(key)[x_columns]
    x
            
    file_clusters[key] = clusters